In [ ]:
from keras.models import load_model
keras_model = load_model('./KerasMnist.h5')

In [ ]:
from coremltools.converters import keras as converter

class_labels = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

mlmodel = converter.convert(keras_model,
                            input_names=['image'],
                            output_names=['digitProbabilities'],
                            class_labels=class_labels,
                            predicted_feature_name='digit')

In [ ]:
coreml_model_path = './MNISTDigitClassifier.mlmodel'
mlmodel.save(coreml_model_path)

In [ ]:
import coremltools

# 作成したモデルの入力形式を確認する
spec = coremltools.utils.load_spec(coreml_model_path)
builder = coremltools.models.neural_network.NeuralNetworkBuilder(spec=spec)

builder.inspect_input_features()

In [ ]:
# 入力の型を28x28のグレースケール画像に変更
from coremltools.proto import FeatureTypes_pb2 as ft
grayscale = ft.ImageFeatureType.ColorSpace.Value('GRAYSCALE')
input_image_type = builder.spec.description.input[0].type.imageType
input_image_type.width = 28
input_image_type.height = 28
input_image_type.colorSpace = grayscale

# 確認
builder.inspect_input_features()

In [ ]:
# save the updated spec
mlmodel_modified = coremltools.models.MLModel(spec)
mlmodel_modified.save('./ModifiedMNISTDigitClassifier.mlmodel')